In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
from Bio import SeqIO

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
UNSPLICED_PATH = DATA_PATH.joinpath('martquery_0219183013_823.txt')
EXON_PATH_0 = DATA_PATH.joinpath('martquery_0225145730_334.txt')
FEATURE_PATH = DATA_PATH.joinpath('mart_export.txt')
UNSPLICED_RESULT_PATH = DATA_PATH.joinpath('gene_symbol_dna_sequence_unspliced.pkl')
EXON_RESULT_PATH = DATA_PATH.joinpath('gene_symbol_dna_sequence_exon.pkl')

In [2]:
df = pd.read_csv(FEATURE_PATH, sep='\t')

df = df[['Gene stable ID', 'Gene name']]

df = df.drop_duplicates()

_df_features = df.copy()

In [3]:
records = []
for datum in SeqIO.parse(UNSPLICED_PATH, "fasta"):
    record = {}
    record['Gene stable ID'] = datum.description
    record['Sequence'] = str(datum.seq)
    records.append(record)

df = pd.DataFrame(records)

df = _df_features.merge(df, how='outer')

assert not df['Gene stable ID'].duplicated().any()

df.to_pickle(UNSPLICED_RESULT_PATH)

_df_unspliced = df.copy()

In [4]:
_df_unspliced = pd.read_pickle(UNSPLICED_RESULT_PATH)

_df_unspliced

,Gene stable ID,Gene name,Sequence
0,ENSMUSG00000064336,mt-Tf,GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACA
1,ENSMUSG00000064337,mt-Rnr1,AAAGGTTTGGTCCTGGCCTTATAATTAATTAGAGGTAAAATTACACATGCAAACCTCCATAGACCGGTGTAAAATCCCTTAAACATTTACTTAAAA...
2,ENSMUSG00000064338,mt-Tv,CATAGTGTAGCTTAATATTAAAGCATCTGGCCTACACCCAGAAGATTTCATGACCAATGAACACTCTGA
3,ENSMUSG00000064339,mt-Rnr2,ACTAATCCTAGCCCTAGCCCTACACAAATATAATTATACTATTATATAAATCAAAACATTTATCCTACTAAAAGTATTGGAGAAAGAAATTCGTAC...
4,ENSMUSG00000064340,mt-Tl1,ATTAGGGTGGCAGAGCCAGGAAATTGCGTAAGACTTAAAACCTTGTTCCCAGAGGTTCAAATCCTCTCCCTAATA
5,ENSMUSG00000064341,mt-Nd1,GTGTTCTTTATTAATATCCTAACACTCCTCGTCCCCATTCTAATCGCCATAGCCTTCCTAACATTAGTAGAACGCAAAATCTTAGGGTACATACAA...
6,ENSMUSG00000064342,mt-Ti,AGAAATATGTCTGATAAAAGAATTACTTTGATAGAGTAAATTATAGAGGTTCAAGCCCTCTTATTTCTA
7,ENSMUSG00000064343,mt-Tq,TAGGATAAGGTGTTTAGGTAGCACGGAGAATTTTGAATTCTTAAGTGTAGGTTCAATTCCTATTGTCCTAG
8,ENSMUSG00000064344,mt-Tm,AGTAAGGTCAGCTAATTAAGCTATCGGGCCCATACCCCGAAAACGTTGGTTTAAATCCTTCCCGTACTA
9,ENSMUSG00000064345,mt-Nd2,ATAAATCCTATCACCCTTGCCATCATCTACTTCACAATCTTCTTAGGTCCTGTAATCACAATATCCAGCACCAACCTAATACTAATATGAGTAGGC...


In [9]:
_df_unspliced['Sequence'].str.len().max()

2960899

In [6]:
records = []
for datum in SeqIO.parse(EXON_PATH_0, "fasta"):
    parts = datum.id.split('|')
    assert len(parts) == 2
    parts.append(str(datum.seq))
    records.append(parts)

df = pd.DataFrame(records, columns=['Gene stable ID', 'Exon stable ID', 'Sequence'])

df = _df_features.merge(df, how='outer')

assert not df['Exon stable ID'].duplicated().any()

df.to_pickle(EXON_RESULT_PATH)

_df_exon = df.copy()

In [7]:
_df_exon = pd.read_pickle(EXON_RESULT_PATH)

_df_exon

,Gene stable ID,Gene name,Exon stable ID,Sequence
0,ENSMUSG00000064336,mt-Tf,ENSMUSE00000521514,GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACA
1,ENSMUSG00000064337,mt-Rnr1,ENSMUSE00000521515,AAAGGTTTGGTCCTGGCCTTATAATTAATTAGAGGTAAAATTACACATGCAAACCTCCATAGACCGGTGTAAAATCCCTTAAACATTTACTTAAAA...
2,ENSMUSG00000064338,mt-Tv,ENSMUSE00000521516,CATAGTGTAGCTTAATATTAAAGCATCTGGCCTACACCCAGAAGATTTCATGACCAATGAACACTCTGA
3,ENSMUSG00000064339,mt-Rnr2,ENSMUSE00000521517,ACTAATCCTAGCCCTAGCCCTACACAAATATAATTATACTATTATATAAATCAAAACATTTATCCTACTAAAAGTATTGGAGAAAGAAATTCGTAC...
4,ENSMUSG00000064340,mt-Tl1,ENSMUSE00000521518,ATTAGGGTGGCAGAGCCAGGAAATTGCGTAAGACTTAAAACCTTGTTCCCAGAGGTTCAAATCCTCTCCCTAATA
5,ENSMUSG00000064341,mt-Nd1,ENSMUSE00000521519,GTGTTCTTTATTAATATCCTAACACTCCTCGTCCCCATTCTAATCGCCATAGCCTTCCTAACATTAGTAGAACGCAAAATCTTAGGGTACATACAA...
6,ENSMUSG00000064342,mt-Ti,ENSMUSE00000521520,AGAAATATGTCTGATAAAAGAATTACTTTGATAGAGTAAATTATAGAGGTTCAAGCCCTCTTATTTCTA
7,ENSMUSG00000064343,mt-Tq,ENSMUSE00000521521,TAGGATAAGGTGTTTAGGTAGCACGGAGAATTTTGAATTCTTAAGTGTAGGTTCAATTCCTATTGTCCTAG
8,ENSMUSG00000064344,mt-Tm,ENSMUSE00000521522,AGTAAGGTCAGCTAATTAAGCTATCGGGCCCATACCCCGAAAACGTTGGTTTAAATCCTTCCCGTACTA
9,ENSMUSG00000064345,mt-Nd2,ENSMUSE00000521523,ATAAATCCTATCACCCTTGCCATCATCTACTTCACAATCTTCTTAGGTCCTGTAATCACAATATCCAGCACCAACCTAATACTAATATGAGTAGGC...


In [10]:
_df_exon['Sequence'].str.len().max()

122802